# Project: Creating a Conversational AI with LangGraph
## Competitor Analysis for Clothing Stores

This notebook implements a Conversational AI pipeline using LangGraph to analyze nearby clothing store competitors, footfall trends, and peak business hours for business intelligence and strategic decision-making.

## Problem Statement

Businesses in competitive locations often need to monitor their market rivals closely to stay relevant and maximize customer engagement. Specifically, clothing stores in bustling areas face intense competition, making it essential to understand the local landscape of competitors, their footfall trends, and peak hours.

This code is designed to address this challenge by creating a conversational AI pipeline using LangGraph that allows users to query and generate a report on nearby clothing store competitors. The system integrates a language model with a search tool to provide real-time data on competitors, including store footfall and busiest times, giving actionable insights to enhance business strategies.

## Intended Beneficiaries

This solution benefits the following groups:

1. **Business Owners and Managers**: Clothing store owners or managers in areas with high competition can use this report to identify key competitors, analyze peak customer hours, and adjust their own strategies to improve market position.

2. **Marketing and Strategy Teams**: Marketing teams responsible for local engagement can leverage insights on footfall trends to optimize promotions during high-traffic periods.

3. **Real Estate and Location Analysts**: Location-based data on competitor footfall and busy hours can assist analysts in recommending ideal store locations for future openings.

4. **Investors and Market Analysts**: Investors considering opportunities in retail can use competitor analysis to assess market saturation, business potential, and potential risks.

In [1]:
# Install required packages
!pip install langgraph langchain-openai duckduckgo-search --quiet


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Libraries

In [2]:
## Import Libraries

import os
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from duckduckgo_search import DDGS
import json

## LLM Configuration

In [3]:
## Define State and Data Structures

@dataclass
class CompetitorAnalysisState:
    """State for the competitor analysis workflow"""
    location: str
    user_query: str
    search_results: List[Dict[str, Any]] = None
    competitor_data: Dict[str, Any] = None
    analysis_report: str = ""
    messages: List[Any] = None
    current_step: str = "initialization"
    
    def __post_init__(self):
        if self.messages is None:
            self.messages = []
        if self.search_results is None:
            self.search_results = []
        if self.competitor_data is None:
            self.competitor_data = {}

## Search Tool

In [4]:
## LLM Configuration

# Set your OpenAI API key before running
# os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'

# Initialize the LLM
try:
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.3)
    print("LLM initialized successfully")
except Exception as e:
    print(f"Error initializing LLM: {e}")
    print("Please ensure you have set your OpenAI API key")

LLM initialized successfully


## Custom Tool: Competitor Analysis

In [5]:
## Search Tool for Competitor Data

@tool
def search_competitor_data(location: str) -> str:
    """
    Search for clothing store competitors, footfall trends, and peak hours in a given location.
    
    Args:
        location: The location to search for competitors
        
    Returns:
        JSON string containing search results
    """
    try:
        with DDGS() as ddgs:
            # Multiple search queries for comprehensive data
            queries = [
                f"clothing stores competitors in {location}",
                f"shopping footfall trends {location} peak hours",
                f"retail competition analysis {location} busiest times",
                f"fashion stores market analysis {location}"
            ]
            
            all_results = []
            for query in queries:
                results = list(ddgs.text(query, max_results=3))
                all_results.extend(results)
            
            # Remove duplicates and format results
            unique_results = []
            seen_titles = set()
            
            for result in all_results:
                if result.get('title') not in seen_titles:
                    unique_results.append({
                        'title': result.get('title', ''),
                        'body': result.get('body', ''),
                        'href': result.get('href', '')
                    })
                    seen_titles.add(result.get('title', ''))
            
            return json.dumps(unique_results, indent=2)
            
    except Exception as e:
        return json.dumps({"error": f"Search failed: {str(e)}"})

## Create Conversational AI Agent

In [6]:
## LangGraph Workflow Nodes

def initialize_analysis(state: CompetitorAnalysisState) -> CompetitorAnalysisState:
    """Initialize the competitor analysis workflow"""
    print(f"🔍 Initializing competitor analysis for: {state.location}")
    state.current_step = "search"
    
    # Add system message
    system_msg = SystemMessage(content=(
        "You are a business intelligence assistant specializing in retail competitor analysis. "
        "Your task is to analyze clothing store competitors, footfall trends, and peak business hours "
        "to provide actionable insights for business owners, marketing teams, and investors."
    ))
    state.messages.append(system_msg)
    
    return state

def search_competitors(state: CompetitorAnalysisState) -> CompetitorAnalysisState:
    """Search for competitor information"""
    print("🔎 Searching for competitor data...")
    state.current_step = "analyze"
    
    try:
        # Perform search
        search_results_json = search_competitor_data.invoke(state.location)
        search_results = json.loads(search_results_json)
        
        if "error" not in search_results:
            state.search_results = search_results
            print(f"✅ Found {len(search_results)} relevant search results")
        else:
            print(f"❌ Search error: {search_results['error']}")
            state.search_results = []
            
    except Exception as e:
        print(f"❌ Search failed: {e}")
        state.search_results = []
    
    return state

def analyze_competitor_data(state: CompetitorAnalysisState) -> CompetitorAnalysisState:
    """Analyze the search results and generate insights"""
    print("📊 Analyzing competitor data...")
    state.current_step = "generate_report"
    
    if not state.search_results:
        state.analysis_report = "Unable to generate analysis due to lack of search data."
        return state
    
    # Create analysis prompt
    analysis_prompt = f"""
    Based on the following search results about clothing stores in {state.location}, 
    provide a comprehensive competitor analysis report for the following stakeholders:
    
    1. Business Owners and Managers
    2. Marketing and Strategy Teams  
    3. Real Estate and Location Analysts
    4. Investors and Market Analysts
    
    Include the following sections in your report:
    
    ## EXECUTIVE SUMMARY
    Brief overview of the competitive landscape
    
    ## COMPETITOR LANDSCAPE
    - Major clothing stores in the area
    - Market positioning and competitive density
    
    ## FOOTFALL TRAFFIC ANALYSIS
    - Peak business hours and days
    - Customer traffic patterns
    - Seasonal trends if mentioned
    
    ## STRATEGIC INSIGHTS
    - Opportunities for differentiation
    - Optimal timing for promotions
    - Market gaps and potential entry points
    
    ## RECOMMENDATIONS
    For each stakeholder group, provide specific actionable recommendations
    
    Search Results:
    {json.dumps(state.search_results, indent=2)}
    
    Format the response as a professional business report with clear headings and bullet points.
    """
    
    try:
        messages = [
            SystemMessage(content="You are an expert retail business analyst providing comprehensive competitor intelligence."),
            HumanMessage(content=analysis_prompt)
        ]
        
        response = llm.invoke(messages)
        state.analysis_report = response.content
        print("✅ Analysis completed successfully")
        
    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        state.analysis_report = f"Analysis failed due to error: {str(e)}"
    
    return state

def finalize_report(state: CompetitorAnalysisState) -> CompetitorAnalysisState:
    """Finalize the analysis report"""
    print("📋 Finalizing competitor analysis report...")
    state.current_step = "completed"
    
    # Add completion message
    completion_msg = AIMessage(content=(
        f"Competitor analysis for {state.location} has been completed. "
        "The report includes insights for business owners, marketing teams, "
        "real estate analysts, and investors."
    ))
    state.messages.append(completion_msg)
    
    return state

## User Interaction

In [7]:
## Build LangGraph Workflow

def create_competitor_analysis_graph():
    """Create the LangGraph workflow for competitor analysis"""
    
    # Initialize the graph
    workflow = StateGraph(CompetitorAnalysisState)
    
    # Add nodes
    workflow.add_node("initialize", initialize_analysis)
    workflow.add_node("search", search_competitors)
    workflow.add_node("analyze", analyze_competitor_data)
    workflow.add_node("finalize", finalize_report)
    
    # Define the workflow edges
    workflow.set_entry_point("initialize")
    workflow.add_edge("initialize", "search")
    workflow.add_edge("search", "analyze")
    workflow.add_edge("analyze", "finalize")
    workflow.add_edge("finalize", END)
    
    # Add memory for conversation context
    memory = MemorySaver()
    
    # Compile the graph
    app = workflow.compile(checkpointer=memory)
    
    return app

# Create the application
try:
    competitor_analysis_app = create_competitor_analysis_graph()
    print("✅ LangGraph workflow created successfully")
except Exception as e:
    print(f"❌ Error creating workflow: {e}")
    competitor_analysis_app = None

✅ LangGraph workflow created successfully


## User Interaction - Run the Analysis

# Initialize the conversational interface
if competitor_analysis_app:
    chat_interface = CompetitorAnalysisChat(competitor_analysis_app)
    
    # Example 1: General Analysis for Koramangala, Bangalore
    location = 'Koramangala, Bangalore'
    
    print("🚀 Starting Competitor Analysis...")
    general_analysis = chat_interface.analyze_location(location)
    
    print("\n" + "="*60)
    print("📊 ANALYSIS SUMMARY")
    print("="*60)
    print(f"Location: {general_analysis.get('location', 'N/A')}")
    print(f"Search Results Found: {general_analysis.get('search_results_count', 0)}")
    print(f"Status: {general_analysis.get('status', 'Unknown')}")
    
    # Example 2: Business Owner Perspective
    print("\n" + "="*60)
    print("🎯 STAKEHOLDER-SPECIFIC INSIGHTS")
    print("="*60)
    
    print("\n👔 BUSINESS OWNER INSIGHTS:")
    business_insights = chat_interface.get_insights_for_stakeholder(location, "business_owner")
    
    # Example 3: Marketing Team Perspective  
    print("\n📢 MARKETING TEAM INSIGHTS:")
    marketing_insights = chat_interface.get_insights_for_stakeholder(location, "marketing_team")
    
    # Example 4: Investor Perspective
    print("\n💰 INVESTOR INSIGHTS:")
    investor_insights = chat_interface.get_insights_for_stakeholder(location, "investor")
    
else:
    print("❌ Cannot run analysis - workflow not initialized")

## Conclusion

This LangGraph-based Conversational AI system provides comprehensive competitor analysis for clothing stores, delivering actionable insights for multiple stakeholder groups:

### Key Features:
- **State-based Workflow**: Uses LangGraph's StateGraph for structured analysis pipeline
- **Multi-stakeholder Insights**: Tailored reports for business owners, marketing teams, real estate analysts, and investors
- **Real-time Data**: Integrates web search for current market information
- **Conversational Interface**: Interactive chat-based analysis system
- **Memory Management**: Persistent conversation context for follow-up queries

### Benefits:
1. **Business Intelligence**: Real-time competitive landscape analysis
2. **Strategic Planning**: Data-driven insights for decision-making
3. **Market Assessment**: Comprehensive location and competitor evaluation
4. **Stakeholder Communication**: Targeted insights for different audience needs

The system demonstrates how LangGraph can be used to build sophisticated conversational AI workflows that go beyond simple Q&A to provide structured, multi-step analysis with state management and memory.